# How to run a semantic skills from file
Now that you're familiar with Kernel basics, let's see how the kernel allows you to run Semantic Skills and Semantic Functions stored on disk. 

A Semantic Skill is a collection of Semantic Functions, where each function is defined with natural language that can be provided with a text file. 

Refer to our [glossary](../../../docs/GLOSSARY.md) for an in-depth guide to the terms.

The repository includes some examples under the [samples](https://github.com/microsoft/semantic-kernel/tree/main/samples) folder.

For instance, [this](../../skills/FunSkill/Joke/skprompt.txt) is the **Joke function** part of the **FunSkill skill**:

```
WRITE EXACTLY ONE JOKE or HUMOROUS STORY ABOUT THE TOPIC BELOW.
JOKE MUST BE:
- G RATED
- WORKPLACE/FAMILY SAFE
NO SEXISM, RACISM OR OTHER BIAS/BIGOTRY.
BE CREATIVE AND FUNNY. I WANT TO LAUGH.
+++++
{{$input}}
+++++
```

Note the special **`{{$input}}`** token, which is a variable that is automatically passed when invoking the function, commonly referred to as a "function parameter". 

We'll explore later how functions can accept multiple variables, as well as invoke other functions.


In the same folder you'll notice a second [config.json](../../skills/FunSkill/Joke/config.json) file. The file is optional, and is used to set some parameters for large language models like Temperature, TopP, Stop Sequences, etc.

```
{
  "schema": 1,
  "type": "completion",
  "description": "Generate a funny joke",
  "completion": {
    "max_tokens": 500,
    "temperature": 0.5,
    "top_p": 0.5
  }
}
```

Given a semantic function defined by these files, this is how to load and use a file based semantic function.

Configure and create the kernel, as usual, loading also the AI backend settings defined in the [Setup notebook](0-AI-settings.ipynb):

In [1]:
#r "nuget: Microsoft.SemanticKernel, 0.18.230725.3-preview"

#!import config/Settings.cs

using Microsoft.SemanticKernel;

var builder = new KernelBuilder();

// Configure AI backend used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();
if (useAzureOpenAI)
    builder.WithAzureChatCompletionService(model, azureEndpoint, apiKey);
else
    builder.WithOpenAIChatCompletionService(model, apiKey, orgId);

IKernel kernel = builder.Build();

Installed Packages Microsoft.SemanticKernel, 0.18.230725.3-preview

Import the skill and all its functions:

In [3]:
// note: using skills from the repo
var skillsDirectory = Path.Combine(System.IO.Directory.GetCurrentDirectory(), "..", "..", "samples", "skills");

var funSkillFunctions = kernel.ImportSemanticSkillFromDirectory(skillsDirectory, "FunSkill");

How to use the skill functions, e.g. generate a joke about "*time travel to dinosaur age*":

In [4]:
var result = await funSkillFunctions["Joke"].InvokeAsync("time travel to dinosaur age");

Console.WriteLine(result);

Why did the dinosaur refuse to wear a seatbelt in his time machine? Because he wanted to feel a little Jurassic!


Great, now that you know how to load a skill from disk, let's show how you can [create and run a semantic function inline.](./03-semantic-function-inline.ipynb)